# Working with Binance API
- https://python-binance.readthedocs.io/en/latest/overview.html

In [1]:
import datetime as dt
import os
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import dateparser
import math
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import coint
from binance.client import Client

from utils import get_closes

In [2]:
API_PUBLIC = os.environ.get("B_PUBLIC_KEY")
API_SECRET = os.environ.get("B_SECRET_KEY")
client = Client(API_PUBLIC, API_SECRET)

In [3]:
ltc_sym = "LTCBTC"
eth_sym = "ETHBTC"
interval = Client.KLINE_INTERVAL_1MINUTE
timedelta = dt.timedelta(minutes=10)
eth_df = get_closes(client, eth_sym, interval, timedelta)
ltc_df = get_closes(client, ltc_sym, interval, timedelta)

eth_lr = get_log_returns(eth_df["c"].values)
ltc_lr = get_log_returns(ltc_df["c"].values)
diff = eth_lr - ltc_lr
hedge_ratio = (eth_df["c"]/ltc_df["c"]).values[1:]

data = np.stack([eth_lr, ltc_lr, diff, hedge_ratio]).T
cols = ["eth", "ltc", "diff", "hedge_ratio"]
ix = eth_df["t1"].values[1:]
df = pd.DataFrame(data=data, columns=cols, index=ix)
df["time_idx"] = list(range(len(df)))

NameError: name 'get_log_returns' is not defined

In [ ]:
eth_df

In [ ]:
# eth_df.to_csv("data/ethbtc_1min.csv")
# ltc_df.to_csv("data/ltcbtc_1min.csv")

In [ ]:
coint

In [ ]:
t1 = 10
t2 = 50
fig, ax1 = plt.subplots(1, 1, sharex=True, figsize=(12,6))
linewidth = 3
alpha = 1
ax1.plot(df.iloc[t1:t2]["eth"],linewidth=linewidth, alpha=alpha, label="eth")
ax1.plot(df.iloc[t1:t2]["ltc"], linewidth=linewidth, alpha=alpha, label="ltc")
ax1.plot(df.iloc[t1:t2]["diff"], "--g", linewidth=linewidth, alpha=alpha, label="eth - ltc")
ax1.legend(loc="best")

# using pytorch forecasting

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
import torch

from pytorch_forecasting import Baseline, NBeats, TimeSeriesDataSet, DeepAR, TemporalFusionTransformer
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_forecasting.data.examples import generate_ar_data
from pytorch_forecasting.metrics import SMAPE, NormalDistributionLoss, QuantileLoss

# build dataset

In [ ]:
ddf = df[["eth", "ltc", "diff", "time_idx"]]
ddf = ddf.set_index("time_idx").reset_index()
ddf["group"] = 0
ddf

In [ ]:
coint_t, p_value, crit_value = coint(ddf["eth"].values, ddf["ltc"].values)

In [ ]:
p_value

In [ ]:
crit_value

In [ ]:
coint?

In [ ]:
# start with just eth returns
ddf = df[["eth", "ltc", "diff", "time_idx"]]
ddf = ddf.set_index("time_idx").reset_index()
ddf["group"] = 0

context_length = 45
prediction_length = 3
training_cutoff = ddf["time_idx"].max() - prediction_length

train_tsds = TimeSeriesDataSet(
    ddf[ddf["time_idx"] <= training_cutoff],
    time_idx = "time_idx",
    target = "diff",
    categorical_encoders={"group": NaNLabelEncoder().fit(ddf.group)},
    group_ids = ["group"],
    time_varying_unknown_reals=["eth", "ltc", "diff"],
    max_encoder_length=context_length,
    max_prediction_length=prediction_length,
    target_normalizer=None,
    add_relative_time_idx=True
)
val_tsds = TimeSeriesDataSet.from_dataset(train_tsds, ddf, min_prediction_idx=training_cutoff + 1)

batch_size = 128
train_dataloader = train_tsds.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = val_tsds.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

In [ ]:
train_tsds.get_parameters()

# Start model building

In [ ]:
# calculate baseline absolute error
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
baseline_predictions = Baseline().predict(val_dataloader)
SMAPE()(baseline_predictions, actuals.view(1,-1))

In [ ]:
pl.seed_everything(42)
trainer = pl.Trainer(gpus=0, gradient_clip_val=0.1)

model_hyper_params = {
    "learning_rate": 3e-2,
    "hidden_size": 16,
    "lstm_layers": 2,
    "attention_head_size": 4,
    "output_size": 7,
    "loss": QuantileLoss(),
    "time_varying_reals_encoder": ["eth", "diff", "ltc"]
}
net = TemporalFusionTransformer.from_dataset(
    train_tsds, 
    **model_hyper_params
)

In [ ]:
# net.summarize("full")
# net.hparams

In [ ]:
# find optimal learning rate
res = trainer.tuner.lr_find(net, train_dataloader=train_dataloader, val_dataloaders=val_dataloader, min_lr=1e-5)
print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()
net.hparams.learning_rate = res.suggestion()

In [ ]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
trainer = pl.Trainer(
    max_epochs=50,
    gpus=0,
    weights_summary="top",
    gradient_clip_val=0.1,
    callbacks=[early_stop_callback],
    limit_train_batches=30,
)

trainer.fit(
    net,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

In [ ]:
best_model_path = trainer.checkpoint_callback.best_model_path
best_model = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

In [ ]:
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
predictions = best_model.predict(val_dataloader)
(actuals - predictions).abs().mean()

In [ ]:
raw_predictions, x = best_model.predict(val_dataloader, mode="raw", return_x=True)

In [ ]:
best_model.plot_prediction(x, raw_predictions, idx=0, add_loss_to_title=True);

In [ ]:
predictions, x = best_model.predict(train_dataloader, return_x=True)
predictions_vs_actuals = best_model.calculate_prediction_actual_by_variable(x, predictions)
best_model.plot_prediction_actual_by_variable(predictions_vs_actuals);

In [ ]:
interpretation = best_model.interpret_output(raw_predictions, reduction="sum")
best_model.plot_interpretation(interpretation)

In [ ]:
dependency = best_model.predict_dependency(
    val_dataloader.dataset, "eth", np.linspace(0, 30, 30), show_progress_bar=True, mode="dataframe"
)

In [ ]:
# plotting median and 25% and 75% percentile
agg_dependency = dependency.groupby("eth").normalized_prediction.agg(
    median="median", q25=lambda x: x.quantile(0.25), q75=lambda x: x.quantile(0.75)
)
ax = agg_dependency.plot(y="median")
ax.fill_between(agg_dependency.index, agg_dependency.q25, agg_dependency.q75, alpha=0.3);